#Dependencies#

In [ ]:
!pip install transformers --quiet
!pip install datasets transformers[SentencePiece] --quiet
!pip install solver --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
import torch

FOLDERNAME = 'CS685-Project/s2s-decipherment-multilingual'
sys.path.append('/content/drive/My Drive/{}/code'.format(FOLDERNAME))
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

%cd /content/drive/My\Drive/$FOLDERNAME

%load_ext autoreload
%autoreload 2

In [4]:
from datasets import *
from transformers import AutoTokenizer, AutoModel, AdamW, DataCollatorWithPadding, get_scheduler, logging
import pandas as pd
import torch
from torch import Tensor
from torch.optim import *
import torch.optim
from torch.utils.data import DataLoader
from torch.nn import *
from torch.nn.functional import one_hot as get_one_hot_enc
from torch.nn.functional import cross_entropy
from torch.nn import Transformer, TransformerEncoder, TransformerEncoderLayer, Embedding, TransformerDecoderLayer, TransformerDecoder
from tqdm.auto import tqdm
import numpy as np
from os import listdir
from os.path import isfile, join
import math
import sys
from lxml.html import fromstring
import lxml.html as PARSER
import string
import random
from collections import Counter

FOLDERNAME = 'CS685-Project/s2s-decipherment-multilingual'
sys.path.append('/content/drive/My Drive/{}/code'.format(FOLDERNAME))
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

from PositionalEncoding import *
from preprocess import *
from models import *
from data_utils import *
from data import *
from debug import *
from train_test import *

device


device(type='cpu')

# **Pre-Reqs: Create Master Data**
Run this only if the data/ folder is empty

In [5]:
_,_,_ = create_master_data('/content/drive/My Drive/{}/data'.format(FOLDERNAME))

NameError: ignored

# **Body**

###Prepare Data###

In [44]:
#Define some constants for training
mode = 'dev'
batch_size = 16
epochs = 1
learning_rate = 3e-5
chr_to_idx = get_chr_to_idx()
idx_to_chr = get_idx_to_chr()

#Get train, test and val DataFrames; they will be useful in error analysis
df_train, df_val = get_split_dfs('/content/drive/My Drive/{}/master_data/{}.csv'.format(FOLDERNAME,mode))
df_test = pd.read_csv('/content/drive/My Drive/{}/master_data/{}.csv'.format(FOLDERNAME,'test'))

#Create Train, Test, and Val Dataloaders

train_dataset = CipherDataset(df_train)
val_dataset = CipherDataset(df_val)
test_dataset = CipherDataset(df_test)
raw_datasets = DatasetDict({'train':train_dataset, 'validation':val_dataset, 'test':test_dataset})
train_dataloader = DataLoader(raw_datasets['train'], batch_size=batch_size)
val_dataloader = DataLoader(raw_datasets['validation'], batch_size=batch_size)
test_dataloader = DataLoader(raw_datasets['test'], batch_size=batch_size)

In [41]:
print(len(train_dataloader))
for batch in train_dataloader:
  for k,v in batch.items():
    print(k)
    print(v.size())
    print(v)

0


###Train and Evaluate Model###

In [11]:
%ls data

catalan.dev    english.dev    german.dev       latin.dev         spanish.dev
catalan.test   english.test   german.test      latin.test        spanish.test
catalan.train  english.train  german.train     latin.train       spanish.train
danish.dev     finnish.dev    hungarian.dev    norwegian.dev     swedish.dev
danish.test    finnish.test   hungarian.test   norwegian.test    swedish.test
danish.train   finnish.train  hungarian.train  norwegian.train   swedish.train
dutch.dev      french.dev     italian.dev      portuguese.dev
dutch.test     french.test    italian.test     portuguese.test
dutch.train    french.train   italian.train    portuguese.train


In [10]:
model = Deciphormer(ntoken=len(get_chr_to_idx()))
optimizer = SGD(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
model.to(device)
train_loss_list, validation_loss_list = fit(model, optimizer, loss_fn, train_dataloader, val_dataloader, 10)

NameError: ignored

# **Model**

In [ ]:
model.train()
batch_size = 32
num_of_batches = lines.shape[0]//batch_size

optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)
loss = torch.nn.CrossEntropyLoss()

for i in range(num_of_batches):
    encoder_input, decoder_tgt, decoder_final,_ = get_batch(lines, tgt, i)

    encoder_input = encoder_input.to(device)
    decoder_tgt = decoder_tgt.to(device)
    decoder_final = decoder_final.to(device)

    model_encoder_out, model_decoder_out = model(encoder_input, decoder_tgt)

    loss_tensor = loss(model_decoder_out,decoder_final)

    optimizer.zero_grad()
    loss_tensor.backward()
    optimizer.step()
    print(i)

0
1
2


In [ ]:
print(lines.shape[0]/32)

2243.125


# **Evaluation**

In [ ]:
eval_src, eval_tgt = get_tensor_file('/content/drive/MyDrive/cs685/project/gutenberg-data/catalan.test')

In [ ]:
rev_alphabet = {}
for key in alphabet_dict.keys():
    rev_alphabet[alphabet_dict[key]] = key

In [ ]:
model.eval()
num_of_batches = eval_src.shape[0]

for i in range(num_of_batches):
    encoder_input, decoder_tgt, decoder_final, target_sentences = get_batch(eval_src, eval_tgt, i, batch_size = 1)
    encoder_input = encoder_input.to(device)
    pred_sentence = ''
    for j in range(max_len):
        decoder_one_hot, decoder_tgt = one_hot_code_vocab([pred_sentence], start = True, end = False)
        decoder_tgt = decoder_tgt.to(device)

        model_encoder_out, model_decoder_out = model(encoder_input, decoder_tgt)
        logits = model_decoder_out.detach().cpu().numpy()
        
        logits = logits.squeeze()

        pred_char = np.argmax(logits[j])
        if(pred_char == end_token):
              break
        elif(pred_char == space_token):
              pred_sentence = ' '
        elif(pred_char < 26):
              pred_sentence += rev_alphabet[pred_char]
    print(pred_sentence)
    print(target_sentences)
    print('-'*20)

startstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstart
['larribada_del_cinque_cosa_dolca_seria_pera_mi_la_seva_vinguda']
--------------------
startstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstart
['sir_tobias_per_deu']
--------------------
startstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstartstart
['si_certament_nhi_tinc_una_pero_que']
--------------------
startstartstartstartstarts